In [ ]:
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Input, Dropout
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator

path = '/home/ubuntu/nbs/state_farm/data/sample/'
batch_size=10

#path = '/home/ubuntu/nbs/state_farm/data/'
#batch_size=64

target_size=(224, 224)


#generate the batches
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
    datagen = ImageDataGenerator()
    return datagen.flow_from_directory(directory=directory,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=shuffle)

train_gen = ImageDataGenerator(rotation_range=0, 
                                     width_shift_range=0, 
                                     height_shift_range=0.1, 
                                     shear_range=0.1, 
                                     zoom_range=0.2, 
                                     channel_shift_range=10,
                                     fill_mode='nearest')

batches = train_gen.flow_from_directory(directory=path+'train_set',  
                                        target_size=target_size,
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=True)

#batches = get_batches(path+'train_set', shuffle=True)

valid_batches = get_batches(path+'valid_set', batch_size=batch_size*2, shuffle=False)

# initialize the model
initial_model = VGG16(weights="imagenet", include_top=True)

#finetuning
x = Dense(batches.num_class, activation='softmax')(initial_model.layers[-2].output)
model = Model(initial_model.input, x)

# we freeze the other layers 
for layer in initial_model.layers: layer.trainable=False

opt = Adam(lr=0.001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit_generator(batches, steps_per_epoch=batches.samples//batch_size, nb_epoch=5,
                validation_data=valid_batches, validation_steps=valid_batches.samples//batch_size)



Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:60: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:60: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=150, validation_steps=100)`


Epoch 1/5
149/150 [============================>.] - ETA: 0s - loss: 2.4421 - acc: 0.3208

In [ ]:
for layer in model.layers[:10]: 
    layer.trainable = False 

for layer in model.layers[10:]: 
    layer.trainable = True
    
opt = SGD(lr=10e-5) 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, steps_per_epoch=batches.samples//batch_size, 
                    nb_epoch=10, validation_data=valid_batches, 
                    validation_steps=valid_batches.samples//batch_size)

In [ ]:
# predictions on new data
test_batches = get_batches(path+'test', batch_size=batch_size)
preds = model.predict_generator(test_batches, steps=test_batches.samples//batch_size)


In [ ]:
preds_clipes = preds.clip(min=0.01, max=0.99)
print(preds_clipes[:2])